<a href="https://colab.research.google.com/github/sandhrabijoy/Pytorch-and-ML-learning/blob/master/pytorch_practicce3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np, pandas as pd, pylab as pl
import h5py,torch,re
from tensorflow import image as timage
from torchvision.datasets import CIFAR10 as tcifar10
from torchvision import transforms,utils
from torch.utils.data import DataLoader as tdl
from torch.utils.data import Dataset as tds
from torch.utils.data.dataset import Subset
import torch.nn as tnn
import torch.utils.checkpoint as tcp
from collections import OrderedDict as od
from IPython.core.magic import register_line_magic
dev= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class TData(tds):
  def __init__(self,X,y):
    self.X=torch.tensor(X,dtype=torch.float32)
    self.y=torch.tensor(y,dtype=torch.int32)
  def __getitem__(self, index):
    train_img,train_lbl=self.X[index],self.y[index]
    return train_img,train_lbl
  def __len__(self):
    return self.y.shape[0]

In [4]:
def load_h5py(path,img_size):
  f=h5py.File(path,'r')
  keys=list(f.keys());print(keys)
  x=np.array(f[keys[1]],dtype='float32')/255
  x=x.reshape(-1,3,img_size,img_size)
  y=np.array(f[keys[2]],dtype='int32')-1
  N=len(y);n=int(.2*N)
  shuffle_ids=np.arange(N)
  np.random.RandomState(23).shuffle(shuffle_ids)
  x,y=x[shuffle_ids],y[shuffle_ids]
  x_test,x_valid,x_train=x[:n],x[n:2*n],x[2*n:]
  y_test,y_valid,y_train=y[:n],y[n:2*n],y[2*n:]

  df=pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],[x_train.dtype,x_valid.dtype,x_test.dtype],[y_train.shape,y_valid.shape,y_test.shape],[y_train.dtype,y_valid.dtype,y_test.dtype]],
                  columns=['train,valid','test'],index=['image shape','image type','label shape','label type'])
  display(df)
  return[[x_train,x_valid,x_test],[y_train,y_valid,y_test]]
